In [1]:
import numpy as np
import copy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from keras import Sequential, Model, layers, callbacks, optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
# from sklearn.model_selection import GridSearchCV
#from keras.wrappers.scikit_learn import KerasClassifier
# from scikeras.wrappers import KerasRegressor, KerasClassifier
# import keras

2024-05-24 01:31:40.970106: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 01:31:40.970155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 01:31:40.971731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 01:31:40.980135: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
milk_df = pd.read_csv("milk.csv", parse_dates=["Date"])
milk_df.dropna(how='any', axis=0, inplace=True, ignore_index=True)
df = milk_df.drop('Date', axis=1)
df = df.drop('dim2', axis=1)
df = df.drop('Cond', axis=1)
df = df.drop('Peak', axis=1)
print(df)

       AnEar  lactation  dim  milk_production  MilkShif    Dur
0        123          4    1             11.5     11.50  276.0
1        123          4    2             48.4     24.20  384.0
2        123          4    3             66.4     33.20  672.0
3        123          4    4             57.1     28.55  354.0
4        123          4    5             78.5     39.25  396.0
...      ...        ...  ...              ...       ...    ...
25841   9977          2  301             72.8     36.40  540.0
25842   9977          2  302             73.0     36.50  600.0
25843   9977          2  303             70.9     35.45  540.0
25844   9977          2  304             70.6     35.30  516.0
25845   9977          2  305             67.7     33.85  522.0

[25846 rows x 6 columns]


In [3]:
dv_treino = {}
dv_teste = {}
nomes_vacas = df['AnEar'].unique()
print(nomes_vacas)
for nome in nomes_vacas:
    vals = df[df['AnEar'] == nome].values
    vals = np.delete(vals, 0, axis=1)
    dv_treino[nome] = vals[:50]
    dv_teste[nome] = vals[50:]


[ 123 1009 1017 1209 1218 1242 1243 1275 1280 1305 1327 1329 1341 1362
 1368 1387 1393 7761 8064 8334 8385 8393 8434 8435 8438 8509 8581 8667
 8723 8732 8741 8932 8947 8996 9005 9066 9069 9078 9081 9090 9101 9149
 9188 9207 9284 9346 9363 9388 9389 9431 9463 9464 9478 9487 9488 9525
 9529 9549 9558 9562 9583 9885 9891 9892 9894 9908 9909 9910 9917 9918
 9920 9922 9926 9928 9929 9933 9939 9943 9944 9946 9951 9961 9967 9968
 9977]


In [4]:
dv_treino_aug = {}
for nome in nomes_vacas:
    new_vals = []
    vals = dv_treino[nome]
    for i in range(0, len(vals)-1):
        new_vals.append(vals[i])
        new_line = [(vi+vf)/2 for vi, vf in zip(vals[i], vals[i+1])]
        new_vals.append(new_line)
    new_vals.append(vals[len(vals)-1])

    dv_treino_aug[nome] = np.array(new_vals)


In [5]:
def get_model(input_size, hidden_layer_sizes, dropout, learning_rate, loss):
  model=Sequential()
  model.add(layers.InputLayer((input_size,)))
  model.add(layers.Flatten())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(hidden_layer_sizes, activation="relu", kernel_initializer="he_uniform"))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(1,activation="linear"))

  #final_model = Model([model.input], out)
  model.compile(loss=loss, optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=['mae'])

  return model

In [6]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-05-24 01:31:43.102542: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-24 01:31:43.114137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-24 01:31:43.117574: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
all_aug_values = np.concatenate(list(dv_treino_aug.values()), axis=0)
x_train_aug = np.delete(all_aug_values, 2, axis=1)
y_train_aug = all_aug_values[:, 2].reshape(-1, 1)

x_aug_mmc = MinMaxScaler().fit(x_train_aug)
x_train_aug = x_aug_mmc.transform(x_train_aug)

y_aug_mmc = MinMaxScaler().fit(y_train_aug)
y_train_aug = y_aug_mmc.transform(y_train_aug)

In [8]:
all_test_values = np.concatenate(list(dv_teste.values()), axis=0)
x_test = np.delete(all_test_values, 2, axis=1)
y_test = all_test_values[:, 2].reshape(-1, 1)

x_aug_teste = x_aug_mmc.transform(x_test)
y_aug_teste = y_aug_mmc.transform(y_test)

In [ ]:
resultados = {
    "hidden_layer_sizes": [], 
    "learnRate": [],
    "dropout": [],
    "batchSize": [],
    "MAE": [], 
    "MSE": [],
}

seed = 101
x_treino, x_valid, y_treino, y_valid = train_test_split(x_train_aug, y_train_aug, test_size=0.3, random_state=seed)
i = 1
for hls in [256, 512, 784]:
    for lr in [0.01, 0.001, 0.0001]:
        for do in [0.3, 0.4, 0.5]:
            for bs in [16, 32]:
                modelo_aug = get_model(input_size=4, hidden_layer_sizes=hls, dropout=do, learning_rate=lr, loss='mean_absolute_error')
                name = "aug"
                hist_aug = modelo_aug.fit(
                    x_treino,
                    y_treino,
                    batch_size=bs,
                    epochs=50,
                    validation_data=(x_valid, y_valid),
                    callbacks=[
                        ModelCheckpoint(filepath=f"pesos/{name}_{hls}_{lr}_{do}_bs.keras",save_best_only=True,verbose=0)
                    ],
                    verbose=0
                )
                pred_aug = modelo_aug.predict(x_aug_teste, verbose=0)
                resultados["MAE"].append(mean_absolute_error(y_aug_teste, pred_aug))
                resultados["MSE"].append(mean_squared_error(y_aug_teste, pred_aug))
                resultados["hidden_layer_sizes"].append(hls)
                resultados["learnRate"].append(lr)
                resultados["dropout"].append(do)
                resultados["batchSize"].append(bs)
                print(f"FIM - {i}/54  -  {hls}  -  {lr}  -  {do}  -  {bs}")
                i+=1

2024-05-24 01:31:43.162482: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-24 01:31:43.166265: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-24 01:31:43.169863: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

FIM - 1/54  -  256  -  0.01  -  0.3  -  16
FIM - 2/54  -  256  -  0.01  -  0.3  -  32
FIM - 3/54  -  256  -  0.01  -  0.4  -  16
FIM - 4/54  -  256  -  0.01  -  0.4  -  32
FIM - 5/54  -  256  -  0.01  -  0.5  -  16
FIM - 6/54  -  256  -  0.01  -  0.5  -  32
FIM - 7/54  -  256  -  0.001  -  0.3  -  16
FIM - 8/54  -  256  -  0.001  -  0.3  -  32
FIM - 9/54  -  256  -  0.001  -  0.4  -  16
FIM - 10/54  -  256  -  0.001  -  0.4  -  32
FIM - 11/54  -  256  -  0.001  -  0.5  -  16
FIM - 12/54  -  256  -  0.001  -  0.5  -  32
FIM - 13/54  -  256  -  0.0001  -  0.3  -  16
FIM - 14/54  -  256  -  0.0001  -  0.3  -  32
FIM - 15/54  -  256  -  0.0001  -  0.4  -  16
FIM - 16/54  -  256  -  0.0001  -  0.4  -  32
FIM - 17/54  -  256  -  0.0001  -  0.5  -  16
FIM - 18/54  -  256  -  0.0001  -  0.5  -  32
FIM - 19/54  -  512  -  0.01  -  0.3  -  16
FIM - 20/54  -  512  -  0.01  -  0.3  -  32
FIM - 21/54  -  512  -  0.01  -  0.4  -  16
FIM - 22/54  -  512  -  0.01  -  0.4  -  32
FIM - 23/54  -  512  - 